Leggere il file GTF dei trascritti umani. Un estratto è gtf.zip

1.    Costruire l'elenco degli esoni distinti (un esone è duplicato se ha stessa posizione di inizio/fine e stesso strand).
2.    Per ogni gene costruire l'elenco dei suoi esoni

Come prima cosa uso la soluzione dell'esercizio 4.3 per leggere i dati.

In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv('../data/extract1.gtf', sep='\t', skiprows=5, 
                 names=["seqname", "source", "feature", "start", "end", "score", "strand", 
                        "frame", "attributes"])
df.head()

,seqname,source,feature,start,end,score,strand,frame,attributes
0,3,havana,gene,58572744,58574319,.,+,.,"gene_id ""ENSG00000243384""; gene_version ""1""; g..."
1,3,havana,transcript,58572744,58574319,.,+,.,"gene_id ""ENSG00000243384""; gene_version ""1""; t..."
2,3,havana,exon,58572744,58572790,.,+,.,"gene_id ""ENSG00000243384""; gene_version ""1""; t..."
3,3,havana,exon,58573477,58573650,.,+,.,"gene_id ""ENSG00000243384""; gene_version ""1""; t..."
4,3,havana,exon,58574191,58574319,.,+,.,"gene_id ""ENSG00000243384""; gene_version ""1""; t..."


In [3]:
nuovi_campi = set()
for a in df['attributes']:
    for field in re.split("; ", a):
        m = re.search("^\S+", field)
        if m:
            nuovi_campi.add(m.group(0))
nuovi_campi

{'ccds_id',
 'exon_id',
 'exon_number',
 'exon_version',
 'gene_biotype',
 'gene_id',
 'gene_name',
 'gene_source',
 'gene_version',
 'havana_gene',
 'havana_gene_version',
 'havana_transcript',
 'havana_transcript_version',
 'protein_id',
 'protein_version',
 'tag',
 'transcript_biotype',
 'transcript_id',
 'transcript_name',
 'transcript_source',
 'transcript_support_level',
 'transcript_version'}

In [4]:
def estrai_nuovo_campo(attributi, nuovo_campo):
    m = re.search(nuovo_campo + ' \"(.*?)\"', attributi)
    if m:
        return m.group(1)
    else:
        return None

In [5]:
for campo in nuovi_campi:
    df[campo] = df["attributes"].apply(lambda x: estrai_nuovo_campo(x, campo))

Questa volta però estraggo le righe che si riferiscono a *exon*

In [7]:
esoni = df.query("feature == 'exon'").copy()
esoni.head()

,seqname,source,feature,start,end,score,strand,frame,attributes,exon_number,...,transcript_biotype,protein_version,gene_id,havana_gene_version,transcript_name,transcript_source,havana_transcript,protein_id,transcript_version,gene_biotype
2,3,havana,exon,58572744,58572790,.,+,.,"gene_id ""ENSG00000243384""; gene_version ""1""; t...",1,...,antisense,None,ENSG00000243384,1,RP11-475O23.2-001,havana,OTTHUMT00000353555,None,1,antisense
3,3,havana,exon,58573477,58573650,.,+,.,"gene_id ""ENSG00000243384""; gene_version ""1""; t...",2,...,antisense,None,ENSG00000243384,1,RP11-475O23.2-001,havana,OTTHUMT00000353555,None,1,antisense
4,3,havana,exon,58574191,58574319,.,+,.,"gene_id ""ENSG00000243384""; gene_version ""1""; t...",3,...,antisense,None,ENSG00000243384,1,RP11-475O23.2-001,havana,OTTHUMT00000353555,None,1,antisense
7,3,havana,exon,58607080,58607129,.,+,.,"gene_id ""ENSG00000244383""; gene_version ""1""; t...",1,...,antisense,None,ENSG00000244383,1,FAM3D-AS1-001,havana,OTTHUMT00000353590,None,1,antisense
8,3,havana,exon,58607682,58607945,.,+,.,"gene_id ""ENSG00000244383""; gene_version ""1""; t...",2,...,antisense,None,ENSG00000244383,1,FAM3D-AS1-001,havana,OTTHUMT00000353590,None,1,antisense


In [9]:
distinti = set()
for row in esoni.itertuples():
    distinti.add(str(row.start) + ':' + str(row.end) + ':' + str(row.strand))
distinti

{'123946803:123947319:-',
 '149966161:149966586:-',
 '139389548:139389670:-',
 '132578796:132578881:-',
 '112199321:112199469:-',
 '142784704:142785287:+',
 '100751552:100751674:-',
 '149752210:149752287:-',
 '134650879:134651636:-',
 '128813327:128813445:+',
 '124702038:124702116:+',
 '72504806:72505089:-',
 '150552356:150552883:+',
 '108414232:108414428:-',
 '119708591:119708674:+',
 '149129638:149130453:+',
 '147405452:147405849:-',
 '102418060:102418207:+',
 '93995874:93995944:+',
 '100630705:100630809:+',
 '128486803:128487076:-',
 '81536911:81537095:-',
 '101994561:101994897:+',
 '112537531:112537890:-',
 '134371000:134371494:-',
 '138694786:138694907:-',
 '123620207:123620336:-',
 '88382585:88382833:+',
 '112991000:112991350:+',
 '140941853:140942095:+',
 '114291102:114291296:+',
 '133700882:133701193:+',
 '151755072:151755131:-',
 '154277598:154277835:-',
 '152301919:152302449:+',
 '89047442:89048434:-',
 '107110592:107110653:-',
 '84893242:84893379:+',
 '121834966:121835033:-'

Calcola gli esoni per ogni gene

In [11]:
def esoni_distinti(df):
    distinti = set()
    for row in df.itertuples():
        distinti.add(str(row.start) + ':' + str(row.end) + ':' + str(row.strand))
    return distinti

In [14]:
esoni.groupby('gene_id').apply(esoni_distinti).head()

gene_id
ENSG00000004399    {129556617:129557034:-, 129589351:129589527:-,...
ENSG00000014257    {132352720:132352788:+, 132328324:132328362:+,...
ENSG00000017260    {130894590:130894775:+, 130955743:130955883:+,...
ENSG00000018408    {149736552:149736714:-, 149520699:149520989:-,...
ENSG00000031081    {119383084:119383226:+, 119382292:119382399:+,...
dtype: object